## Elo Hackathon 2020
### Union Squad –  Daniel Rocha, Letícia Fiorito, Everton Arruda, Juliana Lima
Ideia central: Oferta de cashback por BOT de WhatsApp, SMS ou comando de voz. Ex. “pizza”, e o app mostra opções ao redor (com geocoding) de pizzarias que oferecem cashback para pagamentos com ELO.

Ideia complementar: BOT capta dados de georeferenciamento para dar sugestões e personalizar a oferta de cashback. Dessa forma, é mapeado a territorialidade percorrida pelo consumidor para sugerir cashback em serviços próximos aos locais de trânsito.

Crescimento inicial inorgânico: (isenção de taxas, redução de tarifas) para incentivar aceitação de pagamentos digitais pelo comércio. Após adesão de comerciantes, a contrapartida seria o direcionamento de novos clientes (incentivados pelo cashback) ao estabelecimento, de modo que o restaurante teria que arcar com os custos de manter o cashback (ou optar por cessá-lo).

Cultura cashless: pagamento com QR Code (ex. comanda na pizzaria vem com QR code).

### Importando bibliotecas e fazendo ajustes visuais

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import palettable
import matplotlib.pyplot as plt
%matplotlib inline
from jupyterthemes import jtplot
jtplot.style(theme='monokai', context='notebook', ticks=True, grid=False)

In [2]:
for m,n in zip(["display.max_columns","display.max_rows"], [None,10]):
    pd.set_option(m,n)

import matplotlib.style
import matplotlib as mpl
for m,n in zip(['grid.color','axes.titlecolor','axes.labelcolor','xtick.color','ytick.color','axes.titlesize',
    'figure.figsize','legend.framealpha'],['white','white','white','white','white','20.0','13.0, 8.0','0.8']):
    mpl.rcParams[m] = n

### Importando os dados da API